# 네이버 부동산 크롤링
- 세종시 동별로 구분시켜서 크롤링

In [1]:
from random import randint
from tqdm.notebook import tqdm
from time import sleep
from selenium import webdriver
import datetime
import datetime
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains


path="chromedriver.exe"

##### 동 추가로 설정
- 아래와 같은 형태로 모든 동 다 추가

In [11]:
#동_dict = {'연서면' : 'https://m.land.naver.com/map/36.583436:127.247939:14:3611036000/APT:JGC/A1:B1:B2'}


동_dict = {'고운동' : 'https://m.land.naver.com/map/36.51723:127.236034:14:3611011200/APT:JGC/A1:B1:B2',
           '금남면' : 'https://m.land.naver.com/map/36.462913:127.289505:14:3611034000/APT:JGC/A1:B1:B2',
           '나성동' : 'https://m.land.naver.com/map/36.486603:127.264252:14:3611010700/APT:JGC/A1:B1:B2' ,
           '다정동' : 'https://m.land.naver.com/map/36.493891:127.243908:14:3611010900/APT:JGC/A1:B1:B2'}

#기타 동 추가 가능

##### dictionary type : 
- key
- values

In [12]:
동_dict.keys()

dict_keys(['고운동', '금남면', '나성동', '다정동'])

In [13]:
동_dict.values()

dict_values(['https://m.land.naver.com/map/36.51723:127.236034:14:3611011200/APT:JGC/A1:B1:B2', 'https://m.land.naver.com/map/36.462913:127.289505:14:3611034000/APT:JGC/A1:B1:B2', 'https://m.land.naver.com/map/36.486603:127.264252:14:3611010700/APT:JGC/A1:B1:B2', 'https://m.land.naver.com/map/36.493891:127.243908:14:3611010900/APT:JGC/A1:B1:B2'])

##### 모든 동에 for문을 loop 시킴

In [14]:
len(동_dict)

4

In [16]:
df=pd.DataFrame() # 빈 데이터프레임 생성



#모든 동에 대한 for문
for len_ in tqdm(range(len(동_dict))):

    print('현재 진행 동 : ' , list(동_dict.keys())[len_])
    sei=webdriver.Chrome(path)
    url = list(동_dict.values())[len_]

    sei.get(url)  #동별로 설정해놓은 url 접속
    sleep(3)
    

    sei.find_element_by_xpath('//*[@id="_map"]/div[4]/div/div/div[1]/a[1]').click()
    sei.find_element_by_xpath('//*[@id="_map"]/div[4]/div/div/div[1]/a[2]').click()
    sei.find_element_by_xpath('//*[@id="_map"]/div[4]/div/div/div[1]/a[3]').click()  #매물 확인하기위한 과정,, 큰의미는없음

    ##############################################################################################

    total=[]
    get_num=0

    try:  #아래부턴 거의 같음
        for dot in range(20):  
            sei.find_element_by_xpath('//*[@id="_map"]/div[1]/div/div[1]/div[3]/div[1]/div[1]/div['+str(dot+1)+']/span').click()
            sleep(3)


            lenth = int(sei.find_element_by_xpath('//*[@id="_map"]/div[1]/div/div[1]/div[3]/div[1]/div[1]/div['+str(dot+1)+']/span').text)
            get_num += lenth

            #//*[@id="_listContainer"]/div/div[3]/div/div[1]
            #//*[@id="_listContainer"]/div/div[3]/div/div[2]
            element = sei.find_element_by_xpath('//*[@id="_listContainer"]/div/div[3]')


            #//*[@id="_listContainer"]/div/div[3]/div/div[1]
            if lenth > 20:
                for ww in range(lenth//15):
                    sei.execute_script("arguments[0].scrollTo(0, 1000000);", element)
                    sleep(randint(100,150)/100)



                soup = bs(sei.page_source)
                sleep(randint(100,150)/40)        
                total += soup.find_all("div",class_='item_area _Listitem')
            else:
                sei.execute_script("arguments[0].scrollTo(0, 1000000);", element)
                sleep(randint(100,150)/100)

                soup=bs(sei.page_source)
                sleep(randint(100,150)/40)
                total+=soup.find_all("div",class_='item_area _Listitem')        

        print('get %d datas'%get_num)
    except:
        print("%d 's dots get datas"%dot)

    
    
    ## DB 저장  // 거의 같음
    DBdata=[]

    #print('total : ' , total)
    
    for ww in range(len(total)):

        b = total[ww].find("a",class_="_moreLink")

        if b!= None:
             print("https://m.land.naver.com/article/info/"+b["_articleno"])


        else:
            c = total[ww].find("a",class_="item_link _innerLink")["href"]
            print("https://m.land.naver.com"+c)

현재 진행 동 :  고운동
8 's dots get datas
https://m.land.naver.com/article/info/2067265759
https://m.land.naver.com/article/info/2067198198
https://m.land.naver.com/article/info/2067155016
https://m.land.naver.com/article/info/2067031539
https://m.land.naver.com/article/info/2067147495


TypeError: 'NoneType' object is not subscriptable

257

In [32]:
고운동urlList = ['https://m.land.naver.com/article/info/2067265759',
                 'https://m.land.naver.com/article/info/2067198198',
                 'https://m.land.naver.com/article/info/2067155016',
                 'https://m.land.naver.com/article/info/2067031539',
                 'https://m.land.naver.com/article/info/2067147495']

#연서면urlList = ['### ', '###', 추가 예정..]

In [59]:
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup


for url in 고운동urlList:
    
    print(url)

    sei=webdriver.Chrome(path)
    sei.get(url)  #동별로 설정해놓은 url 접속
    
    html = sei.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    #print(soup)
    
    print(soup.select("#content > div > div.detail_apartment > div.detail_info > div > div.detail_info_important > div.detail_important_sale > strong"))

    
    break

https://m.land.naver.com/article/info/2067265759
[]


In [82]:
soup.select('#app > div > header > div > div.header_main_service > h1')

[]

In [84]:
soup

<html data-user-agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.66 Safari/537.36" lang="ko"><head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="-1" http-equiv="Expires"/>
<meta content="no-cache" http-equiv="Pragma"/>
<meta content="No-Cache" http-equiv="Cache-Control"/>
<meta content="text/html;charset=utf-8" http-equiv="Content-Type"/>
<meta content="text/javascript" http-equiv="Content-Script-Type"/>
<meta content="text/css" http-equiv="Content-Style-Type"/>
<meta content="telephone=no" name="format-detection"/>
<meta content="index,nofollow" name="robots"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no" id="_viewport" name="viewport"/>
<meta contents="always" name="referrer"/>
<title>네이버 부동산</title>
<link href="https://m.naver.com/favicon.ico" rel="icon" type="image/x-icon"/>
<link href="https://m.naver.com/favicon.

In [83]:
soup.find_all(class_ = "detail_general_market")

[]

In [88]:
soup.find_all(class_ = '#ct')

[]

In [87]:
soup.find_all(class_='content content_option gpu-render')

[<section class="content content_option gpu-render" id="ct" role="main">
 <div _prefix="cf_" class="article_box--option _complexFilterBox _prefix _filter_box_wrap">
 <div class="filter_box _multiFilter" filtername="tradTpCd">
 <div class="heading_option"><h2 class="heading">거래유형</h2></div>
 <div class="option_box--quarter"><ul class="option_list _trad" title="내 옵션"><li class="option_item"> <input headernm="매매" id="cf_A1" name="tradTpCd" type="checkbox" value="A1"/> <label class="option_label" for="cf_A1"> <span class="inner"><span class="txt_checked">매매</span></span> </label></li><li class="option_item"> <input headernm="전세" id="cf_B1" name="tradTpCd" type="checkbox" value="B1"/> <label class="option_label" for="cf_B1"> <span class="inner"><span class="txt_checked">전세</span></span> </label></li><li class="option_item"> <input headernm="월세" id="cf_B2" name="tradTpCd" type="checkbox" value="B2"/> <label class="option_label" for="cf_B2"> <span class="inner"><span class="txt_checked">월세</s

In [80]:
soup.select('#content > div > div.detail_apartment > div.detail_transport > div')

[]

In [74]:
soup.find_all(class_ = "body_theme")

[]

In [63]:
soup.select("#app > div > div.header_main_sub > div > div > strong")

[]

##### 저장 잘 되었나 확인

In [6]:
df

,0,1,2,3,4,5,6,동
0,가락10단지이지더원,1014동,8억,아파트140/107㎡,3/15층,남향,효자공인중개사사무소,고운동
1,가락13단지대광로제비앙,1301동,"4억 9,000",아파트80/59㎡,5/15층,남서향,5개 중개소,고운동
2,가락9단지파밀리에,908동,"4억 8,000",아파트84/59㎡,2/15층,남향,16개 중개소,고운동
3,가락9단지파밀리에,905동,"2,000/70",아파트83/59㎡,7/15층,None,4개 중개소,고운동
4,가락11단지중흥S-클래스에듀카운티,1106동,"6억 1,000",아파트99/73㎡,12/24층,남향,3개 중개소,고운동
...,...,...,...,...,...,...,...,...
885,더하이스트(가온마을10단지),1003동,"8억 2,000",아파트116/84㎡,고/29층,남향,세종더하이스트공인중개사사무소,다정동
886,더하이스트(가온마을10단지),1002동,"8억 5,000",아파트113/84㎡,1/10층,남향,복터진공인중개사사무소,다정동
887,더하이스트(가온마을10단지),1007동,"3억 5,000",아파트116/84㎡,중/26층,None,백두부동산공인중개사사무소,다정동
888,더하이스트(가온마을10단지),1001동,11억,아파트113/84㎡,20/20층,None,하이스트 정원 공인중개사무소,다정동


##### column 설정

In [9]:
df.columns = ['아파트','동_','가격','평','층','향','중개소','동']
df

,아파트,동_,가격,평,층,향,중개소,동
0,가락10단지이지더원,1014동,8억,아파트140/107㎡,3/15층,남향,효자공인중개사사무소,고운동
1,가락13단지대광로제비앙,1301동,"4억 9,000",아파트80/59㎡,5/15층,남서향,5개 중개소,고운동
2,가락9단지파밀리에,908동,"4억 8,000",아파트84/59㎡,2/15층,남향,16개 중개소,고운동
3,가락9단지파밀리에,905동,"2,000/70",아파트83/59㎡,7/15층,None,4개 중개소,고운동
4,가락11단지중흥S-클래스에듀카운티,1106동,"6억 1,000",아파트99/73㎡,12/24층,남향,3개 중개소,고운동
...,...,...,...,...,...,...,...,...
885,더하이스트(가온마을10단지),1003동,"8억 2,000",아파트116/84㎡,고/29층,남향,세종더하이스트공인중개사사무소,다정동
886,더하이스트(가온마을10단지),1002동,"8억 5,000",아파트113/84㎡,1/10층,남향,복터진공인중개사사무소,다정동
887,더하이스트(가온마을10단지),1007동,"3억 5,000",아파트116/84㎡,중/26층,None,백두부동산공인중개사사무소,다정동
888,더하이스트(가온마을10단지),1001동,11억,아파트113/84㎡,20/20층,None,하이스트 정원 공인중개사무소,다정동
